In [1]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
with open("../../config.yaml", "r") as f:
    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt
import writing_tools as wt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False

RESULTS = {}

In [2]:
# Output the data for regressions in R

qual_df = dt.get_post_quality_analysis_data()
qual_df.to_parquet(os.path.join(DATA_PATH, "post_quality_analysis_data.parquet"))

quant_df = dt.get_post_quantity_analysis_data()
quant_df.to_parquet(os.path.join(DATA_PATH, "post_quantity_analysis_data.parquet"))

c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:89: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time


In [ ]:
wt.update_results({
    'AvgPostingCost': f"{qual_df['cost'].mean():,.0f}",
    'AvgPostingCostCents': f"{qual_df['cost'].mean()/10:,.0f}",
    "AvgSats48": f"{qual_df['sats48'].mean():,.0f}",
    "AvgComments48": f"{qual_df['comments48'].mean():,.1f}",
    'SdPostingCost': f"{qual_df['cost'].std():,.0f}",
    "SdSats48": f"{qual_df['sats48'].std():,.0f}",
    "SdComments48": f"{qual_df['comments48'].std():,.1f}",
})

{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6'}

In [4]:
res = subprocess.run([R_PATH, LOCAL_PATH + "/src/R/pay-to-post-analysis.R"], check=True, capture_output=True, text=True)
print(res.stdout)


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Thu, Dec 04, 2025 - 1:27:23 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{\textit{Dependent variable:}} \\ 
\cline{2-5} 
\\[-1.8ex] & \multicolumn{4}{c}{log\_sats48} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 log(Posting Cost) & 0.393$^{***}$ & 0.266$^{***}$ & 0.273$^{***}$ & 0.219$^{***}$ \\ 
  & (0.003) & (0.004) & (0.008) & (0.009) \\ 
  & & & & \\ 
 Constant & 2.530$^{***}$ &  &  &  \\ 
  & (0.010) &  &  &  \\ 
  & & & & \\ 
\hline \\[-1.8ex] 
Territory FE & N & Y & Y & Y \\ 
Week FE & N & N & Y & Y \\ 
Territory Owner FE & N & N & N & Y \\ 
Observations & 179,480 & 179,480 & 179,480 & 179,480 \\ 
R$^{2}$ & 0.079 & 0.159 & 0.181 & 0.184 \\ 
Adjusted R$^{2}$ & 0.079 & 0.159 & 0.180 & 0.183 \\ 
Residual Std. Error & 2.2

In [5]:
quantity_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'posts_cost_reg.csv'),
    index_col = 0
)
coef = quantity_reg_coefs.loc['log_cost', 'coef(rquan3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['PostsCostElasticity'] = elast
RESULTS['PostsCostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6'}

In [6]:
quality_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'sats48_cost_reg.csv'),
    index_col = 0
)
coef = quality_reg_coefs.loc['log_cost', 'coef(rzaps3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['Zaps48CostElasticity'] = elast
RESULTS['Zaps48CostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6'}

In [7]:
quality_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'comments48_cost_reg.csv'),
    index_col = 0
)
coef = quality_reg_coefs.loc['log_cost', 'coef(rcomm3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['Comments48CostElasticity'] = elast
RESULTS['Comments48CostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.265',
 'PostsCostDoublingEffect': '16.8',
 'Zaps48CostElasticity': '0.219',
 'Zaps48CostDoublingEffect': '16.4',
 'Comments48CostElasticity': '0.043',
 'Comments48CostDoublingEffect': '3.0',
 'NumItems': '1,121,485',
 'NumPosts': '207,219',
 'NumComments': '914,266',
 'NumUsers': '11,228',
 'NumTerritories': '117',
 'StartDate': 'June 11, 2021',
 'EndDate': 'October 5, 2025',
 'TotalZaps': '146 million',
 'NonCustodialDate': 'January 3, 2025',
 'TotalCost': '13 million',
 'Sats48': '94',
 'Comments48': '90',
 'NumPostingFeeChanges': '199',
 'AvgPostingCost': '51',
 'AvgSats48': '364',
 'AvgComments48': '3.3',
 'SdPostingCost': '241',
 'SdSats48': '6,662',
 'SdComments48': '9.6'}